<a href="https://colab.research.google.com/github/AhmadArrabi/Cat-Generation-using-GANs/blob/main/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy
from PIL import Image
import tensorflow as tf
from tqdm import tqdm
from tensorflow import keras

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
dataset = keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/data/dataset/dataset3",
    label_mode=None,
    image_size=(64, 64),
    batch_size=128
    )

Found 4486 files belonging to 1 classes.


In [ ]:
#images = [file for file in os.listdir('/content/drive/MyDrive/data/dataset/dataset2') if file.endswith(('jpeg', 'png', 'jpg'))]
#i = 0
#for image in images:
#    img = Image.open('/content/drive/MyDrive/data/dataset/dataset2/'+image)
#    img.thumbnail((64,64))
#    name = '/content/drive/MyDrive/data/dataset/dataset22/' + str(i) + '.png'
#    img.save(name, optimize=True, quality=40)
#    i = i+1

In [ ]:
dataset = dataset.map(lambda x: x / 255.)

------------------------------------------------------------------

In [ ]:
discriminator = keras.Sequential(
    [
     keras.Input(shape = (64,64,3)),
     keras.layers.Conv2D(filters=128 ,kernel_size=5, strides=2, padding='same'),
     keras.layers.LeakyReLU(0.2),
     keras.layers.Conv2D(filters=256 ,kernel_size=5, strides=2, padding='same'),
     keras.layers.LeakyReLU(0.2),
     keras.layers.Conv2D(filters=512 ,kernel_size=5, strides=2, padding='same'),
     keras.layers.LeakyReLU(0.2),
     keras.layers.Conv2D(filters=1024 ,kernel_size=5, strides=2, padding='same'),
     keras.layers.LeakyReLU(0.2),
     keras.layers.BatchNormalization(),
     keras.layers.Flatten(),
     keras.layers.Dense(1, activation='sigmoid')
    ]
)
print(discriminator.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 128)       9728      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 512)         3277312   
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 512)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 1024)        1

In [ ]:
latent_dim = 128

generator = keras.Sequential(
    [
     keras.layers.Input(shape=(latent_dim,)),
     keras.layers.Dense(4*4*1024),
     keras.layers.BatchNormalization(),
     keras.layers.Reshape((4,4,1024)),
     keras.layers.Conv2DTranspose(filters = 512, kernel_size=5, strides=2, padding='same'),
     keras.layers.LeakyReLU(0.2),
     keras.layers.Conv2DTranspose(filters = 256, kernel_size=5, strides=2, padding='same'),
     keras.layers.LeakyReLU(0.2),
     keras.layers.Conv2DTranspose(filters = 128, kernel_size=5, strides=2, padding='same'),
     keras.layers.LeakyReLU(0.2),
     keras.layers.Conv2DTranspose(filters = 3, kernel_size=5, strides=2, padding='same', activation='tanh'),
    ]
)
print(generator.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16384)             2113536   
_________________________________________________________________
batch_normalization_1 (Batch (None, 16384)             65536     
_________________________________________________________________
reshape (Reshape)            (None, 4, 4, 1024)        0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 8, 8, 512)         13107712  
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 8, 8, 512)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 16, 16, 256)       3277056   
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 16, 16, 256)      

In [ ]:
gen_opt = keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
disc_opt = keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)

In [ ]:
loss = keras.losses.BinaryCrossentropy()
latent_dim = 128

In [ ]:
with tf.device('/device:GPU:0'):
  for epoch in range(100):
    for idx, real in enumerate(tqdm(dataset)):
      batch_size = real.shape[0]
      random_latent_vector = tf.random.normal(shape=(batch_size, latent_dim))
      fake = generator(random_latent_vector)

      if idx % 100 == 0:
        img = keras.preprocessing.image.array_to_img(fake[0])
        img.save("/content/drive/MyDrive/data/dataset/generated3/part3/generated_img_%03d_%d.jpg" % (epoch, idx))
      
      #Train Discriminator ylog(y') + (1-y)log(1-y') => ylog(D(x)) + (1-y)log(1-D(G(z)))
      with tf.GradientTape() as disc_tape:
        loss_disc_real = loss(tf.ones((batch_size,1)), discriminator(real))
        loss_disc_fake = loss(tf.zeros((batch_size,1)), discriminator(fake))
        loss_disc = (loss_disc_fake + loss_disc_real)
      
      grads = disc_tape.gradient(loss_disc, discriminator.trainable_weights)
      disc_opt.apply_gradients(
          zip(grads, discriminator.trainable_weights)
      )
      
      #Train generator: log(D(G(z)))
      with tf.GradientTape() as gen_tape:
        fake = generator(random_latent_vector)
        output = discriminator(fake)
        loss_gen = loss(tf.ones(batch_size,1), output)
      
      grads = gen_tape.gradient(loss_gen, generator.trainable_weights)
      gen_opt.apply_gradients(
          zip(grads, generator.trainable_weights)
      )

      #training on ~2600 images took 1h 48m
      #training on ~1800 images took 1h 5esh m
      #training on ~4600 images took 2h 20m

100%|██████████| 36/36 [01:21<00:00,  2.28s/it]


In [ ]:
3print(generator.get_weights)

<bound method Model.get_weights of <keras.engine.sequential.Sequential object at 0x7fc56ab559d0>>


# Saving models

In [ ]:
import os.path
if os.path.isfile('/content/drive/MyDrive/data/models/generator.h5') is False:
  generator.save('/content/drive/MyDrive/data/models/generator.h5')

In [ ]:
if os.path.isfile('/content/drive/MyDrive/data/models/dicriminator.h5') is False:
  discriminator.save('/content/drive/MyDrive/data/models/dicriminator.h5')

## Loading Models

In [ ]:
from keras.models import load_model
discriminator = load_model('/content/drive/MyDrive/data/models/dicriminator.h5')
discriminator.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 128)       9728      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 512)         3277312   
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 512)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 1024)        1

In [ ]:
generator = load_model('/content/drive/MyDrive/data/models/generator.h5')
generator.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16384)             2113536   
_________________________________________________________________
batch_normalization_1 (Batch (None, 16384)             65536     
_________________________________________________________________
reshape (Reshape)            (None, 4, 4, 1024)        0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 8, 8, 512)         13107712  
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 8, 8, 512)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 16, 16, 256)       3277056   
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 16, 16, 256)      